# Задание №1 
### №1.1 

In [1]:
import numpy as np
import random

number = 12
surname_lenght = 6
name_lenght = 4

v1 = np.linspace(number, 6.5, surname_lenght)
print(v1)

[12.  10.9  9.8  8.7  7.6  6.5]


### №1.2 

$$ b_{name\_lenght} = b_1 * q^{name\_lenght-1}, \; где \; b_1 = |13 - number| + 2  \Rightarrow \; q = \sqrt[name\_lenght-1]{\frac{b_{name\_lenght}}{b_1}}$$ 

In [2]:
b_n = 100
b_1 = np.abs(13 - number) + 2
q = (b_n / b_1) ** (1/3)

v2 = np.empty((name_lenght,))
v2[0] = b_1
v2[1:] = q
v2 = np.cumprod(v2)

print(v2)

[  3.           9.65489385  31.07232506 100.        ]


### №1.3

In [3]:
random.seed(42)

v = np.concatenate([v1, v2])
v3 = random.sample(list(v), 3)

print(v3)

[10.9, 12.0, 7.6]


# Задание 2
### Вариант 2

In [4]:
s = 'аааааб'

start, finish = 1072, 1103
alf = [chr(i) for i in range(start, finish + 1, 2)]

ind_ = np.prod([2 * alf.index(i) + 1 for i in s if i in alf])
print(ind_)

1


# Задание 3

In [63]:
row, col = 5, 5 
v = list(v1) * (row * col // len(v1)) 
v1 = v + v[:(row * col - len(v))]

if v3[0] > 10:
    m = np.array(v1).reshape(row, col)
else:
    m = np.array(v1).reshape(row, col).transpose()
    
print(m)


import pandas as pd

month = ['January', 'February', 'March', 'April', 'May', 'June',
         'July', 'August', 'September', 'October', 'November', 'December ']

start_index = (number - 1) % 12
print(start_index)

names = []

if start_index > 7:
    names = month[start_index:]
    names += month[:5 - len(names)]
else:
    names = month[start_index : start_index + 5]


m = pd.DataFrame(data = m, index = names, columns = names)

print(m)

m1_col = [i for i in names if i[0] > 'F']

m1 = m.loc[m1_col, m1_col]
print(m1)

det = np.linalg.det(m1.values)
l = np.linalg.eig(m1.values)
diag = m1.values[range(0, m1.shape[0]), range(0, m1.shape[0])]
v1_ = np.dot(m1.values, m1.values)
v2_ = v1_ ** 2

print(v2_)

[[12.  10.9  9.8  8.7  7.6]
 [ 6.5 12.  10.9  9.8  8.7]
 [ 7.6  6.5 12.  10.9  9.8]
 [ 8.7  7.6  6.5 12.  10.9]
 [ 9.8  8.7  7.6  6.5 12. ]]
11
           December   January  February  March  April
December        12.0     10.9       9.8    8.7    7.6
January          6.5     12.0      10.9    9.8    8.7
February         7.6      6.5      12.0   10.9    9.8
March            8.7      7.6       6.5   12.0   10.9
April            9.8      8.7       7.6    6.5   12.0
         January  March
January     12.0    9.8
March        7.6   12.0
[[47733.5104 55319.04  ]
 [33269.76   47733.5104]]


# Задание 4

In [64]:
import pandas as pd
import numpy as np


frame = pd.DataFrame({'digits' : [22, 231, 4534.324], 'boolean' : [True, True, False],
                     'strings' : ['fdsf', 'abracadabra', 'sadas']})

print(frame)
print()

n = 25
print(frame[(frame['digits'] > n) & (frame['boolean'] == True)].strings)

     digits  boolean      strings
0    22.000     True         fdsf
1   231.000     True  abracadabra
2  4534.324    False        sadas

1    abracadabra
Name: strings, dtype: object


# Задание 5
### Вариант 5

In [65]:
file = open('data.rtf', 'r')
data = file.readlines()
data = [line.rstrip().split() for line in data]

mas = np.array(data, dtype=float).reshape(3,3)
print(mas)

mins = mas.min(axis=1)
print(mins)

bo = np.array([~(mas[i,:] > mins[i]) for i in range(0, len(mins))]).reshape(3,3)
print(bo)
s = sum([mas[i, (j+1)%3] ** ((i+j+2)%3 + 1) for i, j in zip(*np.where(bo == True))])

print(s)

[[1. 0. 2.]
 [2. 0. 0.]
 [3. 0. 1.]]
[0. 0. 0.]
[[False  True False]
 [False  True  True]
 [False  True False]]
11.0


# Задание 6
### №6.1
##### Алгоритм кодирования Хаффмана:
1. Построить частотный словарь букв с помощью  Counter
2. Добавить его в очередь с приоритетом 
3. Пока в очереди есть два элемента, достаем два элемента с минимальными частотами и добавляем сумму обратно в очередь
4. Реализуем два класса Leaf и Node
5. Обойдем все дерево, начиная с корня. Если идем в левого потомка, то добавляем 0, правого - 1

In [67]:
import heapq
from collections import Counter, namedtuple


class Node(namedtuple('Node', ['left', 'right'])):
    def walk(self, code, pref):
        self.left.walk(code, pref + '0')
        self.right.walk(code, pref + '1')
        

class Leaf(namedtuple('Leaf', ['count'])):
    def walk(self, code, pref):
        code[self.count] = pref or '0'
        
    
def encode(s):
    
    numb = 0
    queue = []
    for symb, count in Counter(s).items():
        queue.append((count, numb, Leaf(symb)))
        numb += 1
    
    
    heapq.heapify(queue)
    
    numb = len(queue)
    
    while len(queue) > 1:
        count_1, numb_1, left = heapq.heappop(queue)
        count_2, numb_2, right = heapq.heappop(queue)
        heapq.heappush(queue, (count_1 + count_2, numb, Node(left, right)))
        numb += 1
    
    [(_count, numb, root)] = queue
    code = {}
    root.walk(code, '')
    
    return code


s = 'abra'
code = encode(s)
print(code)

line = ''.join(code[i] for i in s)
print(line)

{'a': '0', 'b': '10', 'r': '11'}
010110


### №6.2

In [72]:
k = len(s)
l = len(line)
dictionary = code
code_line = line


q = ''
values = list(dictionary.values())
keys = list(dictionary.keys())
decode_line = ''

for symb in code_line:
    q += symb
    if q in values:
        decode_line += keys[values.index(q)]
        q = ''

print(decode_line)

abra
